<a href="https://colab.research.google.com/github/aidot-kr/AISecurity/blob/master/10_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 짧은 영어 문장을 짧은 프랑스어 문장으로 번역하는 예제

##### 1. 라이브러리 로드 및 변수 초기화

In [6]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # 훈련을 위한 배치 크기
epochs = 100  # 훈련 할 에포크 수
latent_dim = 256  # 인코딩 공간의 잠재 된 차원 수
num_samples = 10000  # 훈련 할 샘플 수

data_path = 'fra-eng/fra.txt' # 데이터 경로

##### 2. 학습에 사용할 데이터 로드

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/My\ Drive/AISecurity

/content/drive/My Drive/AISecurity


In [4]:
ls

 10_Seq2Seq.ipynb
 11_Attention.ipynb
 11_nmt_with_attention_사본.ipynb
 11_nmt_with_attention.ipynb
 12_transformer.ipynb
'13_BERT Classification.ipynb'
 2_TFIDF.ipynb
 3_SimpleCrawling.ipynb
 4_MeCab.ipynb
 4_Pre_Processing.ipynb
 5_OneHotEncode.ipynb
 5_Word2Vec-FastText.ipynb
 6_Similarity.ipynb
 6_TopicModeling_bak.ipynb
 6_TopicModeling.ipynb
 7_ngram.ipynb
 9-1_RNN.ipynb
 9-1_rnn.py
 9-2_LSTM.ipynb
 9-3_GRU.ipynb
 9-4_BLSTM.ipynb
'(bak)(11_attention.ipynb'
'(Bak)12_Transformer.ipynb'
'(bak)13_BERT Classification.ipynb'
 convert_tensorflow_1_to_tensorflow_2_on_google_colab.ipynb
 deu.txt
 eng-fra.txt
 kor.txt
 model.pt
 multi_cased_L-12_H-768_A-12/
 ratings.txt
 report.txt
 Sample.ipynb
 spam.csv
 spam_test.csv
 spam_train.csv
 spa.txt
 temp2.ipynb
 temp.py
 training_checkpoints/


In [8]:
# 데이터 백터화
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:    
    input_text, target_text = line.split('\t')
    # Tab을 시작 문자로 사용하고 \n 은 종료 문자
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

##### 3. 학습 대상 데이터 확인

In [9]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 91
Max sequence length for inputs: 16
Max sequence length for outputs: 59


##### 4. 번역에 사용될 문장들을 3차원 배열(encoder_input_data, decoder_input_data, decoder_target_data)로 초기화

In [10]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# encoder_input_data는 (num_pairs, max_english_sentence_length, num_english_characters) 
# 형태의 3차원 배열로 영어 문장의 one-hot 형식 벡터 데이터로 초기화.
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
# decoder_input_data는 (num_pairs, max_french_sentence_length, num_french_characters)
# 형태의 3차원 배열로 불어 문장의 one-hot형식 벡터 데이터로 초기화.
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
# decoder_target_data는 decoder_input_data와 같지만
# 하나의 time step만큼 offset 됨. 
# decoder_target_data[:, t, :]는 decoder_input_data[:, t + 1, :]와 동일.
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

##### 5. 문장들을 3차원 배열(encoder_input_data, decoder_input_data, decoder_target_data)로 변환.

In [11]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data는 한 번의 단계로 decoder_input_data보다 앞서 있음.
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

##### 6. 기본 LSTM 기반의 encoder 생성

In [12]:
# 입력 시퀀스의 정의와 처리
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# `encoder_outputs`는 버리고 상태(`state_h, state_c`)는 유지
encoder_states = [state_h, state_c]

##### 7. 기본 LSTM 기반의 decoder 생성

In [13]:
# `encoder_states`를 초기 상태로 사용해 decoder를 설정
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# 전체 출력 시퀀스를 반환하고 내부 상태도 반환하도록 decoder를 설정. 
# 학습 모델에서 상태를 반환하도록 하진 않지만, inference에서 사용할 예정.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

##### 8. 기본 LSTM 기반의 Seq2Seq model을 주어진 encoder_input_data와 decoder_input_data로 decoder_target_data를 예측. (해당 model은 teacher forcing을 사용)

In [ ]:
# `encoder_input_data`와 `decoder_input_data`를 `decoder_target_data`로 반환하도록 모델을 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 실행
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# 모델 저장
model.save('s2s.h5')

Epoch 1/100
125/125 [==============================] - 50s 371ms/step - loss: 1.2110 - accuracy: 0.7178 - val_loss: 1.1289 - val_accuracy: 0.6899
Epoch 2/100
125/125 [==============================] - 46s 364ms/step - loss: 0.8720 - accuracy: 0.7645 - val_loss: 0.8946 - val_accuracy: 0.7543
Epoch 3/100
125/125 [==============================] - 48s 388ms/step - loss: 0.7003 - accuracy: 0.8029 - val_loss: 0.7931 - val_accuracy: 0.7699
Epoch 4/100
125/125 [==============================] - 45s 359ms/step - loss: 0.6101 - accuracy: 0.8224 - val_loss: 0.7134 - val_accuracy: 0.7916
Epoch 5/100
 58/125 [============>.................] - ETA: 21s - loss: 0.5712 - accuracy: 0.8326

##### 9. 샘플링 할 encoder, decoder 모델 정의

In [ ]:
# 샘플링 모델 정의
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state = decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

##### 10. model이 정상적으로 작동하는지 확인하기 위한 색인 생성

In [ ]:
# 시퀀스를 다시 해독하기 위한 역방향 조회 토큰 색인
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

##### 11. 문장 Decoding을 위한 함수 (model이 정상적으로 작동하는지 확인을 위한 함수)

In [ ]:
def decode_sequence(input_seq):
    # 상태 벡터로서 입력값을 encode
    states_value = encoder_model.predict(input_seq)

    # 길이가 1인 빈 목표 시퀀스를 생성
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # 대상 시퀀스 첫 번째 문자를 시작 문자로 기재.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # 시퀀스들의 batch에 대한 샘플링 반복(간소화를 위해, 배치 크기는 1로 상정)
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # 토큰으로 샘플링
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # 탈출 조건 : 최대 길이에 도달하거나
        # 종료 문자를 찾을 경우
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # (길이 1인) 목표 시퀀스 업데이트
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # 상태 업데이트
        states_value = [h, c]

    return decoded_sentence

##### 12. encoder_input_data의 샘플을 decoder_target_data의 표본으로 변환 (model이 정상적으로 작동하는지 확인)

In [ ]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)